In [1]:
import rclpy
from rclpy.node import Node
import sys
import random
import numpy as np
import collections
from pathlib import Path
import json
import shutil
import os
from cookiecutter.main import cookiecutter
import math

rclpy.init(args=sys.argv)
node = rclpy.create_node('glasshouse_generate')

In [2]:
def get_param(name, default):
    try:
        value = node.declare_parameter(name, default).value
    except ConnectionRefusedError:
        value = default
    print('param "{}" = "{}"'.format(name, value))
    return value

#Plants variables
SEED = int(get_param('~seed', '1000'))
MODEL_NAME_PREFIX = get_param('~model_name_prefix', 'tomato')
ROW_COUNT = int(get_param('~row_count', '5'))
ROW_LENGTH = int(get_param('~row_length', '5'))
ROW_DIST = float(get_param('~row_dist', '1'))
CROP_DIST = float(get_param('~crop_dist', '1'))
CROP_NUM_IN_A_ROW = int(math.floor(ROW_LENGTH/CROP_DIST + 1))
#GlassHouse Variables
GH_LINES= int(get_param('~gh_lines', '1'))
GH_PER_LINE= int(get_param('~gh_per_lns', '1'))
FARM_SIZE = [(ROW_COUNT), (ROW_LENGTH)]
FARM_NAME = str(FARM_SIZE[0]) + 'mx' + str(FARM_SIZE[1]) + 'm'
WORLD_NAME = get_param('~world_name', FARM_NAME)
#Light parameters
#It indicates how much light is attenuated independently of distance. Higher values mean that the light attenuates less with distance.
LIGHT_CONSTANT = float(get_param('~light_constant', '0.9'))
#It indicates how the intensity of the light decreases linearly as distance increases. A higher value of this variable will result in faster attenuation of light as it moves away.
LIGHT_LINEAR = float(get_param('~light_linear', '0.3'))
#It indicates how the light attenuates quadratically as distance increases. A higher value of this variable will result in even faster attenuation of light as it moves away.
LIGHT_QUADRATIC = float(get_param('~light_quadratic', '0.4'))

param "~seed" = "1000"
param "~model_name_prefix" = "tomato"
param "~row_count" = "5"
param "~row_length" = "5"
param "~row_dist" = "1"
param "~crop_dist" = "1"
param "~gh_lines" = "1"
param "~gh_per_lns" = "1"
param "~world_name" = "5mx5m"
param "~light_constant" = "0.9"
param "~light_linear" = "0.3"
param "~light_quadratic" = "0.4"


In [3]:
OUT_PATH = Path(get_param('~out_path', str(Path().resolve().parent / 'generated/farm_') + WORLD_NAME)).resolve()

#OUT_PATH = Path(get_param('~out_path', str(Path().resolve().parent / 'generated'))).resolve()
WORLD_TEMPLATE = Path(get_param('~world_template', str(Path().resolve().parent / 'templates/tomato_world_gazebo_classic_and_sim'))).resolve()
#Tomato plant
MODEL_TEMPLATE = Path(get_param('~model_template', str(Path().resolve().parent / 'templates/tomato_model_gazebo_classic_and_sim'))).resolve()
#Structure 
STRUCTURE_TEMPLATE = Path(get_param('~structure_template', str(Path().resolve().parent / 'templates/structure_model_gazebo_classic_and_sim'))).resolve()
#Lamp
LAMP_TEMPLATE = Path(get_param('~lamp_template', str(Path().resolve().parent / 'templates/lamp_model_gazebo_classic_and_sim'))).resolve()
#Soil Bed
SOILBED_TEMPLATE = Path(get_param('~soilbed_template', str(Path().resolve().parent / 'templates/soilbed_model_gazebo_classic_and_sim'))).resolve()
#Flower Pot
FLOWERPOT_TEMPLATE= Path(get_param('~flowerpot_template', str(Path().resolve().parent / 'templates/flowerpot_model_gazebo_classic_and_sim'))).resolve()
#Metal 
METAL_TEMPLATE= Path(get_param('~metal_template', str(Path().resolve().parent / 'templates/metal_model_gazebo_classic_and_sim'))).resolve()


shutil.rmtree(OUT_PATH, ignore_errors=True)
np.random.seed(SEED)


param "~out_path" = "/home/paul/dogtooth_ws/src/gz_tomato_farm_generator/generated/farm_5mx5m"
param "~world_template" = "/home/paul/dogtooth_ws/src/gz_tomato_farm_generator/templates/tomato_world_gazebo_classic_and_sim"
param "~model_template" = "/home/paul/dogtooth_ws/src/gz_tomato_farm_generator/templates/tomato_model_gazebo_classic_and_sim"
param "~structure_template" = "/home/paul/dogtooth_ws/src/gz_tomato_farm_generator/templates/structure_model_gazebo_classic_and_sim"
param "~lamp_template" = "/home/paul/dogtooth_ws/src/gz_tomato_farm_generator/templates/lamp_model_gazebo_classic_and_sim"
param "~soilbed_template" = "/home/paul/dogtooth_ws/src/gz_tomato_farm_generator/templates/soilbed_model_gazebo_classic_and_sim"
param "~flowerpot_template" = "/home/paul/dogtooth_ws/src/gz_tomato_farm_generator/templates/flowerpot_model_gazebo_classic_and_sim"
param "~metal_template" = "/home/paul/dogtooth_ws/src/gz_tomato_farm_generator/templates/metal_model_gazebo_classic_and_sim"


In [4]:
def Gener(ROW_LENGTH, CROP_DIST, g, h):
    
    #_____________________DISTANCE_ROWS_CALIBRATION______________________________
    if ROW_LENGTH < 9:
        if CROP_DIST == 1:
            if ROW_LENGTH > 9:
                print("No space for more plants")
            elif ROW_LENGTH <= 9:
                # Bucle
                for x in range(ROW_COUNT):
                    x_pos1, y_pos1, z_pos1 = ((x*ROW_DIST + g * 24.3)), (0*CROP_DIST + h * 11.245), 0.55
                    ########################################################################################################################################### 
                    #Generate metal_name 
                    r4 = random.randint(0, 1000000)
                    metal_name = 'metal_{}'.format((x * ROW_LENGTH) + r4) 
                    #Generate files using Cookiecutter 
                    cookiecutter( 
                        str(METAL_TEMPLATE), 
                        output_dir= str(OUT_PATH), 
                        overwrite_if_exists=True, 
                        no_input=True, 
                        extra_context={ 
                            'world_name': WORLD_NAME, 
                            'metal_name': metal_name, 
                            'models': 'models' 
                        } 
                    ) 
                    models['list'].append({ 
                        'model': metal_name, 
                        'name': metal_name, 
                        'pose': '{} {} -0.34 0 0 -1.6'.format(x_pos1, y_pos1+0.5)  
                    }) 
                    #Resolve directory path 
                    dir_metal= (OUT_PATH / 'models' / WORLD_NAME / metal_name).resolve() 
                    #Add structures using Markers  
                    metal_id= Markers.add_metal(x_pos1, y_pos1+0.5, -0.34) 
                   #######################################################################################################################
                    for y in range(ROW_LENGTH):

                        r = random.randint(0, 1000000)
                        r2 = random.randint(0, 1000000)
                        model_name = 'tomato_{}'.format((x * ROW_LENGTH + y) + r)
                        cookiecutter(str(MODEL_TEMPLATE),
                                     output_dir=str(OUT_PATH),
                                     overwrite_if_exists=True, no_input=True,
                                     extra_context={'world_name': WORLD_NAME, 
                                                    'model_name': model_name, 'models': "models"})

                        x_pos, y_pos, z_pos = ((x*ROW_DIST + g * 24.3)), (y*CROP_DIST + h * 11.245), 0.55
                        
                        models['list'].append({
                            'model': model_name,
                            'name': model_name,
                            'pose': '{} {} {} 0 0 0'.format(x_pos, y_pos, z_pos)
                        })

                        seed = np.random.randint(10000)
                        dir = (OUT_PATH / 'models' / WORLD_NAME / model_name).resolve()
                        dir_blender = (Path.cwd() / '../blender').resolve()
                        blend = str(dir_blender / 'tomato_gen.blend')
                        script = str(dir_blender / 'tomato_gen.py')
                        ! blender $blend --background --python $script -- --model_dir $dir --seed $seed

                        plant_id = Markers.add_plant(x_pos, y_pos, z_pos)
                        with open(dir / 'markers.json') as markers_file:
                            plant_markers = json.load(markers_file)
                            for marker in plant_markers:
                                if marker['marker_type'] == 'FRUIT':
                                    Markers.add_fruit(
                                        marker['translation'][0] + x_pos,
                                        marker['translation'][1] + y_pos,
                                        marker['translation'][2] + z_pos,
                                        plant_id
                                    ) 
                        #FlowerPot
                        ############################FlowerPot#######################################
                        #Generate flowerpot_name 
                        flowerpot_name = 'flowerpot_{}'.format((x * ROW_LENGTH + y) + r) 
                        #Generate files using Cookiecutter     
                        cookiecutter( 
                            str(FLOWERPOT_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 
                                'world_name': WORLD_NAME, 
                                'flowerpot_name': flowerpot_name, 
                                'models': 'models' 
                            } 
                        ) 
                        
                            
                        models['list'].append({ 
                            'model': flowerpot_name, 
                            'name': flowerpot_name, 
                            'pose': '{} {} {} 0 0 -1.60'.format(x_pos, y_pos, 0.50) 
                        }) 
                            
                        #Resolve directory path 
                        dir_flowerpot= (OUT_PATH / 'models' / WORLD_NAME / flowerpot_name).resolve() 
                                
                        #Add structures using Markers 
                        flowerpot_id= Markers.add_flowerpot(x_pos, y_pos, 0.50)
                         

######################################################################################################################################### 
                        #Generate lamp_name 
                        lamp_name = 'lamp_{}'.format((x * ROW_LENGTH + y) + r2)
                        #Generate files using Cookiecutter 
                        cookiecutter( 
                            str(LAMP_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 
                        
                                'world_name': WORLD_NAME,
                                'light_constant':LIGHT_CONSTANT,
                                'light_linear': LIGHT_LINEAR,
                                'light_quadratic': LIGHT_QUADRATIC,
                                'lamp_name': lamp_name, 
                                'models': 'models' 
                            } 
                        ) 
                        models['list'].append({ 
                            'model': lamp_name, 
                            'name': lamp_name, 
                            'pose': '{} {} 2.6 0 3.14159 1.570795'.format(x_pos, y_pos, 0.50)  
                        }) 
                        #Resolve directory path 
                        dir_lamp= (OUT_PATH / 'models' / WORLD_NAME / lamp_name).resolve() 
                        #Add structures using Markers 
                        lamp_id= Markers.add_lamp(x_pos, y_pos, 2.6) 

        ##############################################LAMP##############################################################
                ########################################################################################################################################### 
                        #Generate soilbed_name 
                        r3 = random.randint(0, 1000000)
                        soilbed_name = 'soilbed_{}'.format((x * ROW_LENGTH + y) + r3) 
                        #Generate files using Cookiecutter 
                        cookiecutter( 
                            str(SOILBED_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 
                                'world_name': WORLD_NAME, 
                                'soilbed_name': soilbed_name, 
                                'models': 'models' 
                            } 
                        ) 
                        models['list'].append({ 
                            'model': soilbed_name, 
                            'name': soilbed_name, 
                            'pose': '{} {} 0.4 0 0 0'.format(x_pos, y_pos)  
                        }) 
                        #Resolve directory path 
                        dir_soilbed= (OUT_PATH / 'models' / WORLD_NAME / soilbed_name).resolve() 
                        #Add soilbed using Markers  
                        soilbed_id= Markers.add_soilbed(x_pos, y_pos, 0.4) 
########################################################################################################################################### 


        elif CROP_DIST == 8 or CROP_DIST == 7 or CROP_DIST == 6 or CROP_DIST == 5:
            if ROW_LENGTH > 2:
                print("No space for more plants")
            elif ROW_LENGTH <= 2:
                # Bucle
                for x in range(ROW_COUNT):
                    x_pos1, y_pos1, z_pos1 = ((x*ROW_DIST + g * 24.3)), (0*CROP_DIST + h * 11.245), 0.55
                    ########################################################################################################################################### 
                    #Generate metal_name 
                    r4 = random.randint(0, 1000000)
                    metal_name = 'metal_{}'.format((x * ROW_LENGTH) + r4) 
                    #Generate files using Cookiecutter 
                    cookiecutter( 
                        str(METAL_TEMPLATE), 
                        output_dir= str(OUT_PATH), 
                        overwrite_if_exists=True, 
                        no_input=True, 
                        extra_context={ 
                            'world_name': WORLD_NAME, 
                            'metal_name': metal_name, 
                            'models': 'models' 
                        } 
                    ) 
                    models['list'].append({ 
                        'model': metal_name, 
                        'name': metal_name, 
                        'pose': '{} {} -0.34 0 0 -1.6'.format(x_pos1, y_pos1+0.5)  
                    }) 
                    #Resolve directory path 
                    dir_metal= (OUT_PATH / 'models' / WORLD_NAME / metal_name).resolve() 
                    #Add metal using Markers  
                    metal_id= Markers.add_metal(x_pos1, y_pos1+0.5, -0.34) 
                   #######################################################################################################################
                    for y in range(ROW_LENGTH):

                        r = random.randint(0, 1000000)
                        r2 = random.randint(0, 1000000)
                        model_name = 'tomato_{}'.format((x * ROW_LENGTH + y) + r)
                        cookiecutter(str(MODEL_TEMPLATE),
                                     output_dir=str(OUT_PATH),
                                     overwrite_if_exists=True, no_input=True,
                                     extra_context={'world_name': WORLD_NAME, 
                                                    'model_name': model_name, 'models': "models"})

                        x_pos, y_pos, z_pos = ((x*ROW_DIST + g * 24.3)), (y*CROP_DIST + h * 11.245), 0.55
                        models['list'].append({
                            'model': model_name,
                            'name': model_name,
                            'pose': '{} {} {} 0 0 0'.format(x_pos, y_pos, z_pos)
                        })

                        seed = np.random.randint(10000)
                        dir = (OUT_PATH / 'models' / WORLD_NAME / model_name).resolve()
                        dir_blender = (Path.cwd() / '../blender').resolve()
                        blend = str(dir_blender / 'tomato_gen.blend')
                        script = str(dir_blender / 'tomato_gen.py')
                        ! blender $blend --background --python $script -- --model_dir $dir --seed $seed

                        plant_id = Markers.add_plant(x_pos, y_pos, z_pos)
                        with open(dir / 'markers.json') as markers_file:
                            plant_markers = json.load(markers_file)
                            for marker in plant_markers:
                                if marker['marker_type'] == 'FRUIT':
                                    Markers.add_fruit(
                                        marker['translation'][0] + x_pos,
                                        marker['translation'][1] + y_pos,
                                        marker['translation'][2] + z_pos,
                                        plant_id
                                    ) 
                        #FlowerPot
                        ############################FlowerPot#######################################
                        #Generate flowerpot_name 
                        flowerpot_name = 'flowerpot_{}'.format((x * ROW_LENGTH + y) + r) 
                        #Generate files using Cookiecutter     
                        cookiecutter( 
                            str(FLOWERPOT_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 
                                'world_name': WORLD_NAME, 
                                'flowerpot_name': flowerpot_name, 
                                'models': 'models' 
                            } 
                        ) 
                        
                            
                        models['list'].append({ 
                            'model': flowerpot_name, 
                            'name': flowerpot_name, 
                            'pose': '{} {} {} 0 0 -1.60'.format(x_pos, y_pos, 0.50) 
                        }) 
                            
                        #Resolve directory path 
                        dir_flowerpot= (OUT_PATH / 'models' / WORLD_NAME / flowerpot_name).resolve() 
                                
                        #Add structures using Markers 
                        flowerpot_id= Markers.add_flowerpot(x_pos, y_pos, 0.50)
                         

######################################################################################################################################### 
                        #Generate lamp_name 
                        lamp_name = 'lamp_{}'.format((x * ROW_LENGTH + y) + r2)
                        #Generate files using Cookiecutter 
                        cookiecutter( 
                            str(LAMP_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 
                                'world_name': WORLD_NAME,
                                'light_constant':LIGHT_CONSTANT,
                                'light_linear': LIGHT_LINEAR,
                                'light_quadratic': LIGHT_QUADRATIC,
                                'lamp_name': lamp_name, 
                                'models': 'models' 
                            } 
                        ) 
                        models['list'].append({ 
                            'model': lamp_name, 
                            'name': lamp_name, 
                            'pose': '{} {} 2.6 0 3.14159 1.570795'.format(x_pos, y_pos, 0.50)  
                        }) 
                        #Resolve directory path 
                        dir_lamp= (OUT_PATH / 'models' / WORLD_NAME / lamp_name).resolve() 
                        #Add structures using Markers 
                        lamp_id= Markers.add_lamp(x_pos, y_pos, 2.6) 

        ##############################################LAMP##############################################################
                ########################################################################################################################################### 
                        #Generate soilbed_name 
                        r3 = random.randint(0, 1000000)
                        soilbed_name = 'soilbed_{}'.format((x * ROW_LENGTH + y) + r3) 
                        #Generate files using Cookiecutter 
                        cookiecutter( 
                            str(SOILBED_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 
                                'world_name': WORLD_NAME, 
                                'soilbed_name': soilbed_name, 
                                'models': 'models' 
                            } 
                        ) 
                        models['list'].append({ 
                            'model': soilbed_name, 
                            'name': soilbed_name, 
                            'pose': '{} {} 0.4 0 0 0'.format(x_pos, y_pos)  
                        }) 
                        #Resolve directory path 
                        dir_soilbed= (OUT_PATH / 'models' / WORLD_NAME / soilbed_name).resolve() 
                        #Add structures using Markers  
                        soilbed_id= Markers.add_soilbed(x_pos, y_pos, 0.4) 
###########################################################################################################################################

        elif CROP_DIST == 4 or CROP_DIST == 3:
            if ROW_LENGTH > 3:
                print("No space for more plants")
            elif ROW_LENGTH <= 3:
                # Bucle
                for x in range(ROW_COUNT):
                    x_pos1, y_pos1, z_pos1 = ((x*ROW_DIST + g * 24.3)), (0*CROP_DIST + h * 11.245), 0.55
                    ########################################################################################################################################### 
                    #Generate metal_name 
                    r4 = random.randint(0, 1000000)
                    metal_name = 'metal_{}'.format((x * ROW_LENGTH ) + r4) 
                    #Generate files using Cookiecutter 
                    cookiecutter( 
                        str(METAL_TEMPLATE), 
                        output_dir= str(OUT_PATH), 
                        overwrite_if_exists=True, 
                        no_input=True, 
                        extra_context={ 
                            'world_name': WORLD_NAME, 
                            'metal_name': metal_name, 
                            'models': 'models' 
                        } 
                    ) 
                    models['list'].append({ 
                        'model': metal_name, 
                        'name': metal_name, 
                        'pose': '{} {} -0.34 0 0 -1.6'.format(x_pos1, y_pos1+0.5)  
                    }) 
                    #Resolve directory path 
                    dir_metal= (OUT_PATH / 'models' / WORLD_NAME / metal_name).resolve() 
                    #Add structures using Markers  
                    metal_id= Markers.add_metal(x_pos1, y_pos1+0.5, -0.34) 
                   #######################################################################################################################
                    for y in range(ROW_LENGTH):

                        r = random.randint(0, 1000000)
                        r2 = random.randint(0, 1000000)
                        model_name = 'tomato_{}'.format((x * ROW_LENGTH + y) + r)
                        cookiecutter(str(MODEL_TEMPLATE),
                                     output_dir=str(OUT_PATH),
                                     overwrite_if_exists=True, no_input=True,
                                     extra_context={'world_name': WORLD_NAME, 
                                                    'model_name': model_name, 'models': "models"})

                        x_pos, y_pos, z_pos = ((x*ROW_DIST + g * 24.3)), (y*CROP_DIST + h * 11.245), 0.55

                        models['list'].append({
                            'model': model_name,
                            'name': model_name,
                            'pose': '{} {} {} 0 0 0'.format(x_pos, y_pos, z_pos)
                        })

                        seed = np.random.randint(10000)
                        dir = (OUT_PATH / 'models' / WORLD_NAME / model_name).resolve()
                        dir_blender = (Path.cwd() / '../blender').resolve()
                        blend = str(dir_blender / 'tomato_gen.blend')
                        script = str(dir_blender / 'tomato_gen.py')
                        ! blender $blend --background --python $script -- --model_dir $dir --seed $seed

                        plant_id = Markers.add_plant(x_pos, y_pos, z_pos)
                        with open(dir / 'markers.json') as markers_file:
                            plant_markers = json.load(markers_file)
                            for marker in plant_markers:
                                if marker['marker_type'] == 'FRUIT':
                                    Markers.add_fruit(
                                        marker['translation'][0] + x_pos,
                                        marker['translation'][1] + y_pos,
                                        marker['translation'][2] + z_pos,
                                        plant_id
                                    ) 

                        #FlowerPot
                        ############################FlowerPot#######################################
                        #Generate flowerpot_name 
                        flowerpot_name = 'flowerpot_{}'.format((x * ROW_LENGTH + y) + r) 
                        #Generate files using Cookiecutter     
                        cookiecutter( 
                            str(FLOWERPOT_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 
                                'world_name': WORLD_NAME, 
                                'flowerpot_name': flowerpot_name, 
                                'models': 'models' 
                            } 
                        ) 
                        
                            
                        models['list'].append({ 
                            'model': flowerpot_name, 
                            'name': flowerpot_name, 
                            'pose': '{} {} {} 0 0 -1.60'.format(x_pos, y_pos, 0.50) 
                        }) 
                            
                        #Resolve directory path 
                        dir_flowerpot= (OUT_PATH / 'models' / WORLD_NAME / flowerpot_name).resolve() 
                                
                        #Add structures using Markers 
                        flowerpot_id= Markers.add_flowerpot(x_pos, y_pos, 0.50)
                         

######################################################################################################################################### 
                        #Generate lamp_name 
                        lamp_name = 'lamp_{}'.format((x * ROW_LENGTH + y) + r2)
                        #Generate files using Cookiecutter 
                        cookiecutter( 
                            str(LAMP_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 

                                'world_name': WORLD_NAME,
                                'light_constant':LIGHT_CONSTANT,
                                'light_linear': LIGHT_LINEAR,
                                'light_quadratic': LIGHT_QUADRATIC,
                                'lamp_name': lamp_name, 
                                'models': 'models' 
                            } 
                        ) 
                        models['list'].append({ 
                            'model': lamp_name, 
                            'name': lamp_name, 
                            'pose': '{} {} 2.6 0 3.14159 1.570795'.format(x_pos, y_pos, 0.50)  
                        }) 
                        #Resolve directory path 
                        dir_lamp= (OUT_PATH / 'models' / WORLD_NAME / lamp_name).resolve() 
                        #Add structures using Markers 
                        lamp_id= Markers.add_lamp(x_pos, y_pos, 2.6) 

        ##############################################LAMP##############################################################
                ########################################################################################################################################### 
                        #Generate soilbed_name 
                        r3 = random.randint(0, 1000000)
                        soilbed_name = 'soilbed_{}'.format((x * ROW_LENGTH + y) + r3) 
                        #Generate files using Cookiecutter 
                        cookiecutter( 
                            str(SOILBED_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 
                                'world_name': WORLD_NAME, 
                                'soilbed_name': soilbed_name, 
                                'models': 'models' 
                            } 
                        ) 
                        models['list'].append({ 
                            'model': soilbed_name, 
                            'name': soilbed_name, 
                            'pose': '{} {} 0.4 0 0 0'.format(x_pos, y_pos)  
                        }) 
                        #Resolve directory path 
                        dir_soilbed= (OUT_PATH / 'models' / WORLD_NAME / soilbed_name).resolve() 
                        #Add structures using Markers  
                        soilbed_id= Markers.add_soilbed(x_pos, y_pos, 0.4) 
                  
        elif CROP_DIST == 2:
            if ROW_LENGTH > 5:
                print("No space for more plants")
            elif ROW_LENGTH <= 5:
                # Bucle
                for x in range(ROW_COUNT):
                    
                    x_pos1, y_pos1, z_pos1 = ((x*ROW_DIST + g * 24.3)), (0*CROP_DIST + h * 11.245), 0.55
                    ########################################################################################################################################### 
                    #Generate metal_name 
                    r4 = random.randint(0, 1000000)
                    metal_name = 'metal_{}'.format((x * ROW_LENGTH ) + r4) 
                    #Generate files using Cookiecutter 
                    cookiecutter( 
                        str(METAL_TEMPLATE), 
                        output_dir= str(OUT_PATH), 
                        overwrite_if_exists=True, 
                        no_input=True, 
                        extra_context={ 
                            'world_name': WORLD_NAME, 
                            'metal_name': metal_name, 
                            'models': 'models' 
                        } 
                    ) 
                    models['list'].append({ 
                        'model': metal_name, 
                        'name': metal_name, 
                        'pose': '{} {} -0.34 0 0 -1.6'.format(x_pos1, y_pos1+0.5)  
                    }) 
                    #Resolve directory path 
                    dir_metal= (OUT_PATH / 'models' / WORLD_NAME / metal_name).resolve() 
                    #Add structures using Markers  
                    metal_id= Markers.add_metal(x_pos1, y_pos1+0.5, -0.34) 
                   #######################################################################################################################
                    for y in range(ROW_LENGTH):

                        r = random.randint(0, 1000000)
                        r2 = random.randint(0, 1000000)
                        model_name = 'tomato_{}'.format((x * ROW_LENGTH + y) + r)
                        cookiecutter(str(MODEL_TEMPLATE),
                                     output_dir=str(OUT_PATH),
                                     overwrite_if_exists=True, no_input=True,
                                     extra_context={'world_name': WORLD_NAME, 
                                                    'model_name': model_name, 'models': "models"})

                        x_pos, y_pos, z_pos = ((x*ROW_DIST + g * 24.3)), (y*CROP_DIST + h * 11.245), 0.55

                        models['list'].append({
                            'model': model_name,
                            'name': model_name,
                            'pose': '{} {} {} 0 0 0'.format(x_pos, y_pos, z_pos)
                        })

                        seed = np.random.randint(10000)
                        dir = (OUT_PATH / 'models' / WORLD_NAME / model_name).resolve()
                        dir_blender = (Path.cwd() / '../blender').resolve()
                        blend = str(dir_blender / 'tomato_gen.blend')
                        script = str(dir_blender / 'tomato_gen.py')
                        ! blender $blend --background --python $script -- --model_dir $dir --seed $seed

                        plant_id = Markers.add_plant(x_pos, y_pos, z_pos)
                        with open(dir / 'markers.json') as markers_file:
                            plant_markers = json.load(markers_file)
                            for marker in plant_markers:
                                if marker['marker_type'] == 'FRUIT':
                                    Markers.add_fruit(
                                        marker['translation'][0] + x_pos,
                                        marker['translation'][1] + y_pos,
                                        marker['translation'][2] + z_pos,
                                        plant_id
                                    )
                        #FlowerPot
                        ############################FlowerPot#######################################
                        #Generate flowerpot_name 
                        flowerpot_name = 'flowerpot_{}'.format((x * ROW_LENGTH + y) + r) 
                        #Generate files using Cookiecutter     
                        cookiecutter( 
                            str(FLOWERPOT_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 
                                'world_name': WORLD_NAME, 
                                'flowerpot_name': flowerpot_name, 
                                'models': 'models' 
                            } 
                        ) 
                        
                            
                        models['list'].append({ 
                            'model': flowerpot_name, 
                            'name': flowerpot_name, 
                            'pose': '{} {} {} 0 0 -1.60'.format(x_pos, y_pos, 0.50) 
                        }) 
                            
                        #Resolve directory path 
                        dir_flowerpot= (OUT_PATH / 'models' / WORLD_NAME / flowerpot_name).resolve() 
                                
                        #Add structures using Markers 
                        flowerpot_id= Markers.add_flowerpot(x_pos, y_pos, 0.50)
                         

######################################################################################################################################### 
                        #Generate lamp_name 
                        lamp_name = 'lamp_{}'.format((x * ROW_LENGTH + y) + r2)
                        #Generate files using Cookiecutter 
                        cookiecutter( 
                            str(LAMP_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 
                                
                                'world_name': WORLD_NAME,
                                'light_constant':LIGHT_CONSTANT,
                                'light_linear': LIGHT_LINEAR,
                                'light_quadratic': LIGHT_QUADRATIC,
                                'lamp_name': lamp_name, 
                                'models': 'models' 
                            } 
                        ) 
                        models['list'].append({ 
                            'model': lamp_name, 
                            'name': lamp_name, 
                            'pose': '{} {} 2.6 0 3.14159 1.570795'.format(x_pos, y_pos, 0.50)  
                        }) 
                        #Resolve directory path 
                        dir_lamp= (OUT_PATH / 'models' / WORLD_NAME / lamp_name).resolve() 
                        #Add structures using Markers 
                        lamp_id= Markers.add_lamp(x_pos, y_pos, 2.6) 

        ##############################################LAMP##############################################################
                ########################################################################################################################################### 
                        #Generate soilbed_name 
                        r3 = random.randint(0, 1000000)
                        soilbed_name = 'soilbed_{}'.format((x * ROW_LENGTH + y) + r3) 
                        #Generate files using Cookiecutter 
                        cookiecutter( 
                            str(SOILBED_TEMPLATE), 
                            output_dir= str(OUT_PATH), 
                            overwrite_if_exists=True, 
                            no_input=True, 
                            extra_context={ 
                                'world_name': WORLD_NAME, 
                                'soilbed_name': soilbed_name, 
                                'models': 'models' 
                            } 
                        ) 
                        models['list'].append({ 
                            'model': soilbed_name, 
                            'name': soilbed_name, 
                            'pose': '{} {} 0.4 0 0 0'.format(x_pos, y_pos)  
                        }) 
                        #Resolve directory path 
                        dir_soilbed= (OUT_PATH / 'models' / WORLD_NAME / soilbed_name).resolve() 
                        #Add structures using Markers  
                        soilbed_id= Markers.add_soilbed(x_pos, y_pos, 0.4) 
###########################################################################################################################################

        elif CROP_DIST <= 0 or CROP_DIST >= 9:
            print("No space for more plants")

    elif ROW_LENGTH >= 9:
        print("No space for more plants")




In [5]:
# helper class to build the markers.json
class Markers:
    markers = []
    last_id = 0

    @staticmethod
    def next_id():
        Markers.last_id += 1
        return Markers.last_id

    @staticmethod
    def reset():
        Markers.markers = []


    @staticmethod
    def add_structure(x, y, z):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'STRUCTURE',
            'id': id,
            'translation': [x, y, z]
        })
        return id

    
    @staticmethod
    def add_soilbed(x, y, z):
        id= Markers.next_id()
        Markers.markers.append({
            'marker_type': 'SOILBED',
            'id': id,
            'translation': [x, y, z]
        })
        return id 
    
    @staticmethod
    def add_metal(x, y, z):
        id= Markers.next_id()
        Markers.markers.append({
            'marker_type': 'METAL',
            'id': id,
            'translation': [x, y, z]
        })
        return id 
    
    @staticmethod
    def add_lamp(x, y, z):
        id= Markers.next_id()
        Markers.markers.append({
            'marker_type': 'LAMP',
            'id': id, 
            'translation': [x, y, z],
        })
        return id

    
    @staticmethod
    def add_flowerpot(x,y,z):
        id= Markers.next_id()
        Markers.markers.append({
            'marker_type': 'FLOWERPOT',
            'id': id, 
            'translation': [x, y, z]
        })
        return id
    
    @staticmethod
    def add_plant(x, y, z):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'PLANT',
            'id': id,
            'translation': [x, y, z]
        })
        return id
    

    @staticmethod
    def add_fruit(x, y, z, plant_id):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'FRUIT',
            'id': id,
            'translation': [x, y, z],
            'plant_id': plant_id
        })
        return id
    
    @staticmethod
    def dumps():
        return json.dumps(Markers.markers, indent=4)

In [6]:

def GenerateGlassHouse(g, h):
    ################### ROW_LENGTH_CALIBRATION ############################
    if ROW_COUNT <= 19:
        if ROW_DIST == 1:
            print("Continue")
            Gener(ROW_LENGTH, CROP_DIST, g, h)
        
        elif ROW_DIST == 2:
            if ROW_COUNT > 10:
                print("No space for more plants")
            elif ROW_COUNT <= 10:
                print("Continue")
                Gener(ROW_LENGTH, CROP_DIST, g, h)
        
        elif ROW_DIST == 3:
            if ROW_COUNT > 7:
                print("No space for more plants")
            elif ROW_COUNT <= 7:
                print("Continue")
                Gener(ROW_LENGTH, CROP_DIST, g, h)
        
        elif ROW_DIST == 4:
            if ROW_COUNT > 5:
                print("No space for more plants")
            elif ROW_COUNT <= 5:
                print("Continue")
                Gener(ROW_LENGTH, CROP_DIST, g, h)
        
        elif ROW_DIST == 5 or ROW_DIST == 6:
            if ROW_COUNT > 4:
                print("No space for more plants")
            elif ROW_COUNT <= 4:
                print("Continue")
                Gener(ROW_LENGTH, CROP_DIST, g, h)
        
        elif ROW_DIST == 7 or ROW_DIST == 8:
            if ROW_COUNT > 3:
                print("No space for more plants")
            elif ROW_COUNT <= 3:
                print("Continue")
                Gener(ROW_LENGTH, CROP_DIST, g, h)
        
        elif 9 <= ROW_DIST <= 18:
            if ROW_COUNT > 2:
                print("No space for more plants")
            elif ROW_COUNT <= 2:
                print("Continue")
                Gener(ROW_LENGTH, CROPDIST, g, h)
    
    elif ROW_COUNT >= 19:
        print("No space for more plants")




In [7]:
# Reset models 
models = {'list': []} 

Markers.reset() 

for g in range (GH_PER_LINE):
    for h in range(GH_LINES):
        structure_name = 'structure_{}'.format(g*GH_LINES+h)  
        cookiecutter( 
            str(STRUCTURE_TEMPLATE), 
            output_dir=str(OUT_PATH), 
            overwrite_if_exists=True, 
            no_input=True, 
            extra_context={ 
            'world_name': WORLD_NAME, 
            'structure_name': structure_name, 
            'models': "models" })
        
        xs_pos, ys_pos, zs_pos= g*24.3+9, h*11.245+4, 0
        
        models['list'].append({ 
            'model': structure_name, 
            'name': structure_name,  
            'pose': '{} {} {} 0 0 0'.format(xs_pos, ys_pos, zs_pos)
        }) 

        dir_path = (OUT_PATH / 'models' / WORLD_NAME / structure_name).resolve() 

 
        structure_id = Markers.add_structure(xs_pos, ys_pos, zs_pos) 

        
        #We add the glass house generaton function
        #GenerateGlassHouse(g, h)
        #Gener(CROP_NUM_IN_A_ROW, ROW_COUNT, g, h)
        GenerateGlassHouse(g, h)
        
        


Continue
Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Read prefs: /home/paul/.config/blender/3.0/config/userpref.blend
Color management: scene view "Filmic" not found, setting default "Standard".
Read blend: /home/paul/dogtooth_ws/src/gz_tomato_farm_generator/blender/tomato_gen.blend
Collada export to: /home/paul/dogtooth_ws/src/gz_tomato_farm_generator/generated/farm_5mx5m/models/5mx5m/tomato_273752/meshes/tomato.dae
Info: Exported 6 Objects
Info: Exported 6 Objects

Blender quit
Error: Not freed memory blocks: 7, total unfreed memory 0.001244 MB
Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Read prefs: /home/paul/.config/blender/3.0/config/userpref.blend
Color management: scene view "Filmic" not found, setting default "Standard".
Read blend: /home/paul/dogtooth_ws/src/gz_tomato_farm_generator/blender/tomato_gen.blend
Collada export t

In [8]:
cookiecutter(str(WORLD_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'models': models, 'farm_area_x': str(FARM_SIZE[0]), 
                            'farm_area_y': str(FARM_SIZE[1]), 'worlds': "worlds"})


'/home/paul/dogtooth_ws/src/gz_tomato_farm_generator/generated/farm_5mx5m/worlds'

In [9]:
with open(OUT_PATH / 'models' / WORLD_NAME / 'markers.json', 'w') as outfile:
    json.dump(Markers.markers, outfile, indent=4, sort_keys=True)